In [27]:
from sklearn import neighbors
from sklearn.preprocessing import Normalizer, OneHotEncoder, StandardScaler, LabelEncoder, normalize

from keras.models import Sequential
from keras.layers import Dense, Activation, Reshape, Merge, Embedding, BatchNormalization, Dropout
from keras.callbacks import ModelCheckpoint
from keras.utils import plot_model

import pandas as pd
import numpy as np
np.random.seed(123)

import pickle, sys, csv, os
from datetime import datetime

In [2]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
sess = tf.Session(config=config) 

NameError: name 'tf' is not defined

# 1. Preprocessing: M04A

In [3]:
r_df = pd.read_csv('Route15.csv', index_col=0)
r_df['TT_30'] = r_df['TT'].shift(-6)

In [4]:
cols = """['W_TT',
 'S_TT',
 'SpdAvg0',
 'occ0',
 'SpdAvg1',
 'occ1',
 'SpdAvg2',
 'occ2',
 'SpdAvg3',
 'occ3',
 'SpdAvg4',
 'occ4',
 'SpdAvg5',
 'occ5',
 'SpdTrnd0',
 'SpdTrnd1',
 'SpdTrnd2',
 'SpdTrnd3',
 'SpdTrnd4',
 'SpdTrnd5',
 'Evt0',
 'Evt1',
 'Evt2',
 'Evt3',
 'Evt4',
 'Evt5',
 'Lanes0',
 'Lanes1',
 'Lanes2',
 'Lanes3',
 'Lanes4',
 'Lanes5',
 'hldy_seq',
 'weekday',
 'timeslot',
 'TT_mean']""".replace('[', '').replace(']','').replace("'",'').replace('\n ','').split(',')

In [62]:
train_df, val_df, test_df = r_df[288*517:288*790], r_df[288*790:288*(790+31)], r_df[288*(790+31):288*(790+31+30)]

In [63]:
test_df

,TT,flow,sVF,bVF,year,month,holiday,hldy_seq,weekday,timeslot,...,Closed3,Closed4,Closed5,Ramp0,Ramp1,Ramp2,Ramp3,Ramp4,Ramp5,TT_30
2017-04-01 00:00:00,148,130,105,25,2017,4,4,1,6,0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,148.0
2017-04-01 00:05:00,149,148,132,16,2017,4,4,1,6,1,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,149.0
2017-04-01 00:10:00,146,119,99,20,2017,4,4,1,6,2,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,144.0
2017-04-01 00:15:00,148,122,101,21,2017,4,4,1,6,3,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,145.0
2017-04-01 00:20:00,148,134,98,36,2017,4,4,1,6,4,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,141.0
2017-04-01 00:25:00,146,120,96,24,2017,4,4,1,6,5,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,147.0
2017-04-01 00:30:00,148,108,82,26,2017,4,4,1,6,6,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,149.0
2017-04-01 00:35:00,149,119,94,25,2017,4,4,1,6,7,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,145.0
2017-04-01 00:40:00,144,100,85,15,2017,4,4,1,6,8,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,148.0
2017-04-01 00:45:00,145,96,73,23,2017,4,4,1,6,9,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,144.0


In [6]:
train_df_x, train_df_y = train_df.drop('TT_30', 1), train_df[['TT_30']]
val_df_x, val_df_y = val_df.drop('TT_30', 1), val_df[['TT_30']]
train_df_x = train_df_x[cols]
val_df_x = val_df_x[cols]

In [7]:
cats = ['Evt0','Evt1', 'Evt2', 'Evt3', 'Evt4', 'Evt5', 'Lanes0', 'Lanes1', 'Lanes2', 'Lanes3', 'Lanes4', 'Lanes5',
 'hldy_seq', 'weekday', 'timeslot']
nums = list(set(cols) - set(cats))

In [8]:
train_df_cat_x = train_df_x[cats]
train_df_num_x = train_df_x[nums]
val_df_cat_x = val_df_x[cats]
val_df_num_x = val_df_x[nums]

In [9]:
train_df_cat_x = train_df_cat_x.fillna('-99')
val_df_cat_x = val_df_cat_x.fillna('-99')
all_df_cat_x = r_df[cats].fillna('-99')

In [10]:
train_df_num_x = train_df_num_x.replace([-99, '-99'], -1).fillna(-1)
val_df_num_x = val_df_num_x.replace([-99, '-99'], -1).fillna(-1)

In [11]:
for cat in cats:
    train_df_cat_x[cat] = train_df_cat_x[cat].astype('int').astype('str')
    val_df_cat_x[cat] = val_df_cat_x[cat].astype('int').astype('str')
    all_df_cat_x[cat] = all_df_cat_x[cat].astype('int').astype('str')

In [12]:
train_cat_x = train_df_cat_x.values.copy().astype('int')
val_cat_x = val_df_cat_x.values.copy().astype('int')
all_cat_x = all_df_cat_x.values.copy().astype('int')
train_num_x = train_df_num_x.values.copy().astype('int')
val_num_x = val_df_num_x.values.copy().astype('int')

In [13]:
train_y = train_df_y.values.copy().flatten()
val_y = val_df_y.values.copy().flatten()

# 1.1 Preprocessing: VD

In [14]:
# route15
root = 'vd_to_g2g'
vd_df = None
for folder in os.listdir(root):
    for file in os.listdir('/'.join([root, folder])):
        if vd_df is None:
            vd_df = pd.read_csv('/'.join([root, folder, file]), header=[0,1], index_col=0)
        else:
            vd_df = vd_df.append(pd.read_csv('/'.join([root, folder, file]), header=[0,1], index_col=0))
            
# for col in vd_df.columns:
#     vd_df[col] = vd_df.apply()

In [15]:
for col in vd_df.columns:
    vd_df[col] = vd_df[col].apply(np.round, args=(2,))

In [16]:
les = []
for i in range(train_df_cat_x.shape[1]):
    le = LabelEncoder()
    le.fit(all_cat_x[:, i])
    les.append(le)
    train_cat_x[:, i] = le.transform(train_cat_x[:, i])
    val_cat_x[:,i] = le.transform(val_cat_x[:, i])

In [17]:
train_vd_df_x, val_vd_df_x = vd_df[:288*(700-517)], vd_df[288*(700-517):]
train_vd_df_x = train_vd_df_x.fillna(-99).replace([-99], -1)
val_vd_df_x = val_vd_df_x.fillna(-99).replace([-99], -1)

In [56]:
vd_df

vd_id               nfbVD-N1-S-45.250-M-LOOP                              \
feature_name                         max_spd avg_spd last_spd total_flow   
2016-06-01 00:00:00                     77.2   60.23     76.0       22.0   
2016-06-01 00:05:00                     35.4   28.00      1.0       11.0   
2016-06-01 00:10:00                     46.8   41.00     14.0       16.0   
2016-06-01 00:15:00                     25.2   19.04      1.0       12.0   
2016-06-01 00:20:00                     41.6   34.80     15.0       13.0   
2016-06-01 00:25:00                     67.2   49.51     91.0       19.0   
2016-06-01 00:30:00                     63.0   49.75      1.0       14.0   
2016-06-01 00:35:00                     52.0   40.76     85.0       15.0   
2016-06-01 00:40:00                     44.4   38.05     17.0       12.0   
2016-06-01 00:45:00                     60.2   47.53     77.0       12.0   
2016-06-01 00:50:00                     35.8   22.98     72.0       12.0   
2016-06-01 00:55:00                     20.2   14.60     13.0        8.0   
2016-06-01 01:00:00                     18.6   18.80      0.0        8.0   
2016-06-01 01:05:00                     24.4   21.58      1.0       13.0   
2016-06-01 01:10:00                     20.4   16.96     12.0        9.0   
2016-06-01 01:15:00                     48.2   36.43      1.0       15.0   
2016-06-01 01:20:00                     48.2   44.25     73.0       10.0   
2016-06-01 01:25:00                     39.6   25.15     11.0       11.0   
2016-06-01 01:30:00                     36.8   25.14     13.0       13.0   
2016-06-01 01:35:00                     16.4   10.24      1.0        9.0   
2016-06-01 01:40:00                     48.8   27.05     92.0       11.0   
2016-06-01 01:45:00                     51.2   39.96     82.0       11.0   
2016-06-01 01:50:00                     21.6   21.17     90.0        4.0   
2016-06-01 01:55:00                     51.6   45.77     79.0       15.0   
2016-06-01 02:00:00                     36.2   24.43      1.0        8.0   
2016-06-01 02:05:00                     21.2   22.10      0.0        8.0   
2016-06-01 02:10:00                      5.6    5.60     12.0        5.0   
2016-06-01 02:15:00                     34.4   21.90     84.0        9.0   
2016-06-01 02:20:00                     19.0   13.00      1.0        7.0   
2016-06-01 02:25:00                      1.0    1.00      1.0        6.0   
...                                      ...     ...      ...        ...   
2016-12-31 21:30:00                    106.4  100.10    103.0      174.0   
2016-12-31 21:35:00                    105.0   96.44    101.0      147.0   
2016-12-31 21:40:00                    112.0  103.34    114.0      174.0   
2016-12-31 21:45:00                    109.0  101.37    113.0      212.0   
2016-12-31 21:50:00                    115.6  104.02    119.0      195.0   
2016-12-31 21:55:00                    111.4  107.99    109.0      177.0   
2016-12-31 22:00:00                    111.2  104.13    101.0      160.0   
2016-12-31 22:05:00                    111.8  105.90    111.0      180.0   
2016-12-31 22:10:00                    114.2  106.57    112.0      157.0   
2016-12-31 22:15:00                    116.0  106.46    112.0      192.0   
2016-12-31 22:20:00                    112.2  103.24    105.0      218.0   
2016-12-31 22:25:00                    111.2  102.30    108.0      176.0   
2016-12-31 22:30:00                    115.4  108.44    110.0      183.0   
2016-12-31 22:35:00                    112.6  108.29    110.0      114.0   
2016-12-31 22:40:00                    111.6  106.17    113.0      139.0   
2016-12-31 22:45:00                    113.6  106.44    113.0      190.0   
2016-12-31 22:50:00                    115.2  110.00    107.0       97.0   
2016-12-31 22:55:00                    117.0  113.14    122.0      117.0   
2016-12-31 23:00:00                    115.0  108.11    111.0      165.0   
2016-12-31 23:05:00                    118.2  10

In [18]:
train_vd_x = train_vd_df_x.values
val_vd_x = val_vd_df_x.values

In [19]:
# to do: encode vd data
# train_vd_df = vd_df.copy()
# for i in range(vd_df.shape[1]):
#     le = LabelEncoder()
#     le.fit(vd_df[:, i])
train_vd_x = train_vd_df_x.values
val_vd_x = val_vd_df_x.values

In [20]:
def split_features(X):
    X_list = []
    for i in range(len(X[0,:])):
        X_list.append(X[:, i])
    return X_list

In [21]:
class Model(object):
    def evaluate(self, X_val, y_val):
        assert(min(y_val) > 0)
        guessed_sales = self.guess(X_val)
        relative_err = np.absolute((y_val - guessed_sales) / y_val)
        result = np.sum(relative_err) / len(y_val)
        return result

In [51]:
class NN_with_EntityEmbedding(Model):

    def __init__(self, X_train, y_train, X_val, y_val, n_cats):
        super().__init__()
        self.data_shape = X_train.shape
        self.n_cats = n_cats
        self.unique_values = []
        for i in range(n_cats):
            self.unique_values.append(len(set(X_train[:, i])))
        self.longest_embedding = 50
        self.nb_epoch = 100
        self.checkpointer = ModelCheckpoint(filepath="best_model_weights.hdf5", verbose=1, save_best_only=True)
        self.max_log_y = max(np.max(np.log(y_train)), np.max(np.log(y_val)))
        self.__build_keras_model()
        self.fit(X_train, y_train, X_val, y_val)
        

    def preprocessing(self, X):
        X_list = split_features(X)
        return X_list

    def __build_keras_model(self):
        models = []
        
        for i in self.unique_values:
            model = Sequential()
            model.add(Embedding(i+1, min(i+1, self.longest_embedding), input_length=1))
            model.add(Reshape(target_shape=(min(i+1, self.longest_embedding), )))
            models.append(model)
            
        print(len(models))
        for _ in range(self.n_cats, self.data_shape[1]):
            model = Sequential()
            model.add(Dense(1, input_dim=1))
            models.append(model)
            
        print(len(models))
        
        self.model = Sequential()
        self.model.add(Merge(models, mode='concat'))
        self.model.add(Dense(1000, kernel_initializer='uniform'))
        self.model.add(BatchNormalization())
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.5))        
        self.model.add(Dense(500, kernel_initializer='uniform'))
        self.model.add(BatchNormalization())        
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(500, kernel_initializer='uniform'))
        self.model.add(BatchNormalization())        
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(500, kernel_initializer='uniform'))
        self.model.add(BatchNormalization())        
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.5))         
        self.model.add(Dense(1))
        self.model.add(BatchNormalization())        
        self.model.add(Activation('sigmoid'))

        self.model.compile(loss='mean_absolute_error', optimizer='adam')

    def _val_for_fit(self, val):
        val = np.log(val) / self.max_log_y
        return val

    def _val_for_pred(self, val):
        return np.exp(val * self.max_log_y)

    def fit(self, X_train, y_train, X_val, y_val):
        self.model.fit(self.preprocessing(X_train), self._val_for_fit(y_train),
                       validation_data=(self.preprocessing(X_val), self._val_for_fit(y_val)),
                       epochs=self.nb_epoch, batch_size=128,
                       # callbacks=[self.checkpointer],
                       )
        # self.model.load_weights('best_model_weights.hdf5')
        print("Result on validation data: ", self.evaluate(X_val, y_val))

    def guess(self, features):
        features = self.preprocessing(features)
        result = self.model.predict(features).flatten()
        return self._val_for_pred(result)

In [52]:
train_num_all = np.hstack((train_num_x, train_vd_x))
val_num_all = np.hstack((val_num_x, val_vd_x))

In [53]:
normalizer = Normalizer()
normalizer.fit(train_num_all)

Normalizer(copy=True, norm='l2')

In [54]:
train_all = np.hstack((train_cat_x, normalizer.transform(train_num_all)))
val_all = np.hstack((val_cat_x, normalizer.transform(val_num_all)))
t1, t2 = train_all, val_all

In [55]:
model = NN_with_EntityEmbedding(t1, train_y, t2, val_y, n_cats=len(cats))

15
78


C:\Anaconda3\envs\gf\lib\site-packages\ipykernel_launcher.py:40: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Train on 52704 samples, validate on 8928 samples
Epoch 1/100
52704/52704 [==============================] - 17s 324us/step - loss: 0.0638 - val_loss: 0.0287
Epoch 2/100
52704/52704 [==============================] - 12s 237us/step - loss: 0.0254 - val_loss: 0.0166
Epoch 3/100
52704/52704 [==============================] - 12s 237us/step - loss: 0.0189 - val_loss: 0.0143
Epoch 4/100
52704/52704 [==============================] - 13s 238us/step - loss: 0.0162 - val_loss: 0.0133
Epoch 5/100
52704/52704 [==============================] - 12s 237us/step - loss: 0.0166 - val_loss: 0.0127
Epoch 6/100
52704/52704 [==============================] - 13s 238us/step - loss: 0.0147 - val_loss: 0.0150
Epoch 7/100
52704/52704 [==============================] - 12s 237us/step - loss: 0.0144 - val_loss: 0.0124
Epoch 8/100
52704/52704 [==============================] - 13s 239us/step - loss: 0.0141 - val_loss: 0.0135
Epoch 9/100
52704/52704 [==============================] - 12s 230us/step - loss: 0.013

Epoch 76/100
52704/52704 [==============================] - 12s 232us/step - loss: 0.0044 - val_loss: 0.0058
Epoch 77/100
52704/52704 [==============================] - 12s 231us/step - loss: 0.0044 - val_loss: 0.0055
Epoch 78/100
52704/52704 [==============================] - 12s 232us/step - loss: 0.0044 - val_loss: 0.0057
Epoch 79/100
52704/52704 [==============================] - 12s 233us/step - loss: 0.0044 - val_loss: 0.0056
Epoch 80/100
52704/52704 [==============================] - 12s 234us/step - loss: 0.0044 - val_loss: 0.0056
Epoch 81/100
52704/52704 [==============================] - 12s 234us/step - loss: 0.0044 - val_loss: 0.0056
Epoch 82/100
52704/52704 [==============================] - 12s 235us/step - loss: 0.0044 - val_loss: 0.0064
Epoch 83/100
52704/52704 [==============================] - 12s 235us/step - loss: 0.0044 - val_loss: 0.0056
Epoch 84/100
52704/52704 [==============================] - 12s 234us/step - loss: 0.0043 - val_loss: 0.0061
Epoch 85/100
52704/

In [29]:
m.unique_values

[7, 7, 7, 7, 7, 7, 6, 6, 6, 5, 6, 6, 7, 7, 288]